In [1]:
import re
import nltk
from nltk.corpus import stopwords
from gensim import corpora,similarities,models
import csv

In [2]:
ls = []
with open("./data/articles.csv",encoding="utf8") as csvfile:
    rows = csv.reader(csvfile)
    next(rows)
    for row in rows:
        ls.append(row)
texts = [text[5].lower() for text in ls]

In [3]:
from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer(r'\w+')
texts_tokenization=[]
for word in texts:
    texts_tokenization.append(tokenizer.tokenize(word))

In [4]:
#stemming
from nltk.stem import WordNetLemmatizer as WL
wl=WL()
texts_lemm = []
i=0     
for part in enumerate(texts_tokenization):
    texts_lemm.append([])
    for word in nltk.pos_tag(texts_tokenization[part[0]]):
        if(word[1][0] == "V"):
            texts_lemm[i].append(wl.lemmatize(word[0],pos='v'))
        else:
            texts_lemm[i].append(word[0])
    i+=1

In [5]:
#stopword
from nltk.corpus import stopwords
filters = stopwords.words('english')
texts_stoped=[]
j=0
for part in enumerate(texts_lemm):
    texts_stoped.append([])
    for word in texts_lemm[part[0]]:
        if word not in filters:
            texts_stoped[j].append(word)
    j+=1

In [6]:
#生成 Lsi model
from gensim import corpora,similarities,models

In [7]:
#把每個字轉成id 
dictionary = corpora.Dictionary(texts_stoped)
print(dictionary)

Dictionary(18588 unique tokens: ['000', '100', '16', '1990s', '2000']...)


In [8]:
#print(texts_stoped[0])
#print([dictionary.token2id[l] for l in texts_stoped[0]])

In [9]:
#計算每一個詞出現頻率 並轉成bow(稀疏向量形式)
corpus = [dictionary.doc2bow(l) for l in texts_stoped]
#print(corpus[0])

In [10]:
#把bow輸入進tfidf
tfidf = models.TfidfModel(corpus)

In [11]:
#查看每個文檔的字詞 score
corpus_tfidf = tfidf[corpus]
# for doc in corpus_tfidf[:1]:
#     print(doc)

In [12]:
#lsi淺在語意索引
lsi = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=100)

In [13]:
#輸入一串 query 型態為string。回傳依分數排序過後的推薦
def recommend(query):
    query_bow = dictionary.doc2bow([word for word in tokenizer.tokenize(query)])
    lsi_vec = lsi[query_bow]
    index = similarities.MatrixSimilarity(lsi[corpus]) 
    sims = index[lsi_vec] 
    sorted_index = sorted(enumerate(sims), key=lambda item: -item[1])
    return sorted_index

In [14]:
query = "I want to know Auto-drive" #使用者輸入
print("推薦給你: ")

for ind,score in recommend(query)[:3]:
    print("title: "+ls[ind][4]+"\nlink: "+ls[ind][3]+"\n推薦分數: "+str(score)+"\n")

推薦給你: 
title: Self-driving cars and the Trolley problem – Tanay Jaipuria – Medium
link: https://medium.com/@tanayj/self-driving-cars-and-the-trolley-problem-5363b86cb82d?source=tag_archive---------4----------------
推薦分數: 0.68157464

title: 73 Mind-Blowing Implications of a Driverless Future
link: https://medium.com/@DonotInnovate/73-mind-blowing-implications-of-a-driverless-future-58d23d1f338d?source=tag_archive---------1----------------
推薦分數: 0.55251926

title: An augmentation based deep neural network approach to learn human driving behavior
link: https://chatbotslife.com/using-augmentation-to-mimic-human-driving-496b569760a9?source=tag_archive---------1----------------
推薦分數: 0.5304466

